In [1]:
import networkx as nx
import pandas as pd

In [2]:
%reload_kedro
catalog = context.catalog

2021-08-24 10:51:10,611 - root - INFO - Registered hooks from 2 installed plugin(s): kedro-mlflow-0.7.3
2021-08-24 10:51:10,706 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2021-08-24 10:51:11,001 - root - INFO - ** Kedro project Spotify Recommendations
2021-08-24 10:51:11,002 - root - INFO - Defined global variable `context`, `session` and `catalog`
2021-08-24 10:51:11,012 - root - INFO - Registered line magic `run_viz`


In [3]:
# split_mpd_pids_to_tids = catalog.load("split_mpd_pids_to_tids")
mpd_track_features = catalog.load("mpd_track_features")
cleaned_mpd_playlists = catalog.load("cleaned_mpd_playlists")

2021-08-24 10:51:11,033 - kedro.io.data_catalog - INFO - Loading data from `mpd_track_features` (ParquetDataSet)...
2021-08-24 10:51:11,202 - kedro.io.data_catalog - INFO - Loading data from `cleaned_mpd_playlists` (ParquetDataSet)...


In [4]:
import dask.dataframe as dd
import dask.distributed as d_dist
from dask.dataframe.core import repartition
from dask.distributed import Client

In [5]:
client = d_dist.client._get_global_client() or Client()

distributed.diskutils - INFO - Found stale lock file and directory '/Users/i854336/Documents/PERSONAL/spotify_playlists/spotify-recommender-training/notebooks/dask-worker-space/worker-31_okb8_', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/i854336/Documents/PERSONAL/spotify_playlists/spotify-recommender-training/notebooks/dask-worker-space/worker-lmi_2leq', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/i854336/Documents/PERSONAL/spotify_playlists/spotify-recommender-training/notebooks/dask-worker-space/worker-o_moc8w0', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/i854336/Documents/PERSONAL/spotify_playlists/spotify-recommender-training/notebooks/dask-worker-space/worker-9j6x4m6p', purging


In [6]:
playlist_feature_cols = [
    "pid",
    "name",
    "collaborative",
    "modified_at",
    "num_followers",
    "num_edits"
]

In [7]:
# cleaned_mpd_playlists = cleaned_mpd_playlists.astype({"name": "string", "collaborative" : "bool"})

In [8]:
def str2bool(v): # Note the self to allow it to be in a class
    return v.lower() in ('yes', 'true', 't', '1', 'yea', 'verily')  # lower() is a method

In [9]:
def convert_text_bool_to_python_bool(val):
    return str2bool(val)

In [10]:
cleaned_mpd_playlists["collaborative"] = (
    cleaned_mpd_playlists["collaborative"]
    .map(convert_text_bool_to_python_bool)
)

In [11]:
cleaned_mpd_playlists["modified_at"] = (
    cleaned_mpd_playlists["modified_at"]
    .map(pd.to_datetime)
)

In [12]:
def convert_feature_row_to_network_x(row, id_col):
    _id = row[id_col]
    _features = row.drop(id_col).to_dict()
    return (_id, _features)

In [13]:
test_row = cleaned_mpd_playlists[playlist_feature_cols].loc[0].compute().infer_objects()

In [14]:
test_row

,pid,name,collaborative,modified_at,num_followers,num_edits
0,0,Throwbacks,False,1970-01-01 00:00:01.493424000,1,6
0,1000,disney,False,1970-01-01 00:00:01.457827200,1,4
0,2000,party party,False,1970-01-01 00:00:01.446854400,1,3
0,3000,Birthday Party,False,1970-01-01 00:00:01.485993600,1,4
0,4000,skate,True,1970-01-01 00:00:01.432252800,1,55
...,...,...,...,...,...,...
0,995000,MY FAVE,False,1970-01-01 00:00:01.507507200,1,46
0,996000,worship music,False,1970-01-01 00:00:01.428710400,1,18
0,997000,christmas,False,1970-01-01 00:00:01.481760000,1,7
0,998000,LIT,False,1970-01-01 00:00:01.481587200,1,5


In [15]:
test_ro = test_row.iloc[0]

In [16]:
test_ro

pid                                       0
name                             Throwbacks
collaborative                         False
modified_at      1970-01-01 00:00:01.493424
num_followers                             1
num_edits                                 6
Name: 0, dtype: object

In [17]:
network_x_test = convert_feature_row_to_network_x(test_ro, "pid")

In [18]:
network_x_test

(0,
 {'name': 'Throwbacks',
  'collaborative': False,
  'modified_at': Timestamp('1970-01-01 00:00:01.493424'),
  'num_followers': 1,
  'num_edits': 6})

In [19]:
# mpd_playlist_features = cleaned_mpd_playlists[playlist_feature_cols]